In [41]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import train_test_split, GridSearchCV

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Loading data

In [24]:
df = pd.read_csv(r'C:\Users\Anastasia\Documents\GitHub2\ophthalmic_drugs\Data\corneal\cleaned_data.csv', usecols = [0,2])
df.head(5)

,SMILES,logPerm
0,CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)...,5.135798
1,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O,5.347108
2,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)N,5.393628
3,C(C(CO)O)O,3.806662
4,CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O,4.442651


Converting to MACCS fp

In [25]:
smiles = df['SMILES'].to_list()

In [26]:
mols = [Chem.MolFromSmiles(i) for i in smiles]
MACCS_list = []
header = ['bit' + str(i) for i in range(167)]
for i in range(len(mols)):
    ds = list(MACCSkeys.GenMACCSKeys(mols[i]).ToBitString())
    MACCS_list.append(ds)
df2 = pd.DataFrame(MACCS_list,columns=header)
df2.insert(loc=0, column='smiles', value=smiles)
df2.head(3)

,smiles,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,...,bit157,bit158,bit159,bit160,bit161,bit162,bit163,bit164,bit165,bit166
0,CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)...,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0
1,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0
2,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)N,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0


Delete high correlated features 

In [27]:
df2 = df2.drop(df2.columns[[12, 34, 48, 50, 52, 56, 57, 59, 60, 61, 62, 64, 65, 68, 70, 71, 72, 74, 77, 81, 82, 89, 95, 103, 106, 107, 108, 111, 120, 125, 131, 135, 136, 144, 148]], axis=1)

In [28]:
df2.head(5)

,smiles,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,...,bit157,bit158,bit159,bit160,bit161,bit162,bit163,bit164,bit165,bit166
0,CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)...,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0
1,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0
2,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)N,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
3,C(C(CO)O)O,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
4,CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0


In [37]:
X = df2.iloc[:, 1:-1].astype(int)

In [30]:
X.shape

(120, 131)

In [32]:
y = df['logPerm']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [39]:
model =  XGBRegressor(random_state=10)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=10, ...)

In [40]:
y_pred = model.predict(X_test)

In [42]:
r2 = r2_score(y_test, y_pred)
r2

0.7700143322020816

### XGBRegressor hyperparameter optimization
Using dataset with MACCS fp and reduced to 131 features by Pearson correlation method

In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 6, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7, 1],
        'n_estimators': [100, 200, 500],
        'colsample_bytree': [0.5, 0.7, 1],
        'objective': ['reg:squaredlogerror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           scoring = 'neg_mean_squared_error',  #MSE
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [45]:
hyperParameterTuning(X_train, y_train)

Fitting 10 folds for each of 1215 candidates, totalling 12150 fits


{'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squaredlogerror',
 'subsample': 1}

In [46]:
optimized_model = XGBRegressor(colsample_bytree = 1,
                                learning_rate = 0.1,
                                max_depth = 3,
                                n_estimators = 100,
                                min_child_weight = 1,
                                objective = 'reg:squarederror',
                                subsample = 1)

In [47]:
optimized_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [48]:
optimized_model.score(X_test, y_test)

0.7089224892645273

Defaults hyperparameters show the best results

### Save the model

In [49]:
import pickle 

In [50]:
pkl_filename = "corneal.pkl"

In [51]:
with open(pkl_filename, 'wb') as file: 
    pickle.dump(model, file)